# Introduction
This notebook was created by Garrett R. Dowdy (gdowdy3@gmail.com) to ease the process of scheduling meetings between visitors and faculty during the ChemE Recruiting Weekends.

The notebook consists of a series of gray boxes ("cells") containing code.
The cells are divided up into sections by bold headers  -- "Introduction", "Setup", and so on.
Many of these headers are accompanied by explanatory text.

To run this code:

1. Verify that the "Visitor Preferences.csv" and "Faculty Availability.csv" files have been uploaded to the same JuliaBox folder as the .pynb file that you clicked to open this notebook. Note: the spelling of the .csv files must be *exact*.
2. On the menu bar above, select **Cell** --> **All Output** --> **Clear**
3. Moving down the document, run each cell in sequence,  reading the explanatory text and checking for errors as you go.
4. Check the summary statistics printed by the final cell.
5. Check the output csv files added to the current JuliaBox folder (see previous tab in your browser).
6. If you are happy with the results, great!  If not, tweak the objective weights in the "Define the Objective Function" section and the parameters in the "Define the Constraints" section, and start over, returning to Step 2.
7. You can exit this notebook gracefully by selecting **File** --> **Close and Halt**.

A cell is run by clicking in the gray box and then pressing [ctrl] + [enter] on your keyboard. 
You will know that a cell is executed when the "In[ ]" to its left is replaced with "In[#]", where "#" is a number. 
"In[\*]" indicates that the cell is currently executing.  You can use the up and down arrow keys to move between cells.

In [ ]:
# UPDATE LOG
# Iteration    Date          Changes
#-------------------------------------------------------------------------------------------------------------------------------
# 1            22-Feb-2017    First attempt at code.
#
# 2            24-Feb-2017    Added the objective of maximizing the minimum happiness.
#                             Added the objective that a meeting with *someone* is better than a meeting with no one.
#
# 3            26-Feb-2017    Changed the mapping between the choice rank and happiness points so that more points are given to the top choices.
#                             Added a faculty-focused output.
#                             Changed "Student Preferences" to "Visitor Preferences" for consistency.
#
# 4            14-Mar-2017    Changed the expected format of the "Faculty Availability" file so that it has higher resolution.
#                             Reverted back to a straight-forward happiness "score" instead of happiness fraction, because the fraction had some weird behavior when I varied the min/max free time periods.
#
# 5             9-Apr-2017    Changed the maximum on the number of free periods to a minimum number of faculty meetings

# Potential Improvements
# - Add automatic detection and correction of NA rows.
# - Add automatic minimizing the gap between largest and smallest number of meetings

# Set Up
## Add Packages
The code makes use of several "packages".  Before using these packages we have to add them.  You don't have to do this every time you run the code, just once, the first time you run the code.

In [ ]:
# #Add packages as necessary
# Pkg.update()
# Pkg.add("JuMP")
# Pkg.add("Clp")
# Pkg.add("DataFrames")

## Define Helper Functions
The code makes use of some custom-made functions. So that they don't disrupt the flow of logic in the body of the code below, it is best to define them here.

If, when running the cells in this section, you see a warning that looks something like

>Warning: Method definition [function name] in module Main at In[#] overwritten...

that's totally fine.
It just means that you defined the function multiple times, unnecessarily.

In [ ]:
function prefPoints(choice::Int64)
    # This function assigns a specific "happiness point" value to the meeting with a visitor's (choice)th choice faculty
    #
    # Input:
    #  choice = a positive integer specify the ranking of a faculty member in a student's mind.  For example, if Prof. Barton is visitor Johnny's first choice, then choice = 1.  Furthermore, if Prof. Braatz is Johnny's second choice, then choice = 2, and so on.
    #
    # Output:
    #  points = the happiness point value associated with giving Johnny a meeting with his (choice)th choice professor.
    
    # calculate the number of points
    points = (10 - choice + 1)^2
    
    return points
end

# Read in the Problem Data
There are two sources of data which define the problem:

1. A csv file describing the visitors: "Visitor Preferences.csv".
2. A csv file describing the faculty: "Faculty Availability.csv".

In this section of the code, these two data files are imported and interpreted.

Executing the following two cells displays the top and bottom portions of the interviewer information input table.  Check the display to make sure there isn't any funny business going on.
In particular, make sure that the very last row of each displayed table isn't entirely "NA NA ... NA", because this will cause problems.
If you see this, go to the appropriate .csv file and delete the entire row corresponding to the "NA NA ... NA" values.
It's OK if "NA" appears at random places in the display.
You just want to make sure you don't have an entire row of "NA".

## Read in the faculty availability

In [ ]:
# Read in the faculty availability
using DataFrames, DataArrays
raw = readtable("Faculty Availability.csv")

In [ ]:
DataFrames.tail(raw)

In [ ]:
# describe the input table
nameCol = 1
firstAvailabilityCol = 2
lastAvailabilityCol = 9

# count the number of faculty
n_F = size(raw,1)
F = 1:n_F

# extract the faculty namese
facultyNames = raw[:,nameCol]

# count the number of time periods
n_T = lastAvailabilityCol - firstAvailabilityCol + 1
T = 1:n_T

# define the subset of time periods that overlap with the TG
T_TG = (n_T - 1):n_T

# initialize the availability matrix
A = zeros(Int64,n_F,n_T)

# extract each faculty member's availability
for f in F
    for t in T
        if typeof(raw[f,firstAvailabilityCol + t - 1])== Int64 && raw[f, firstAvailabilityCol + t - 1] == 1
            A[f,t] = 1
        end
    end
end

## Read in the visitor preferences
The last cell in this section will check each faculty name given in "Visitor Preferences.csv" and make sure that this name also appears in "Faculty Availability.csv".
If the name is not found, the code will print out a warning.

In [ ]:
# Read in the student preferences
raw = readtable("Visitor Preferences.csv")

In [ ]:
DataFrames.tail(raw)

In [ ]:
# describe the input table
nameCol = 1;
firstPrefCol = 2;

# count the number of volunteers
n_V = size(raw,1);
V = 1:n_V

# extract the visitors' names
visitorNames = Array{String,1}(n_V);
for v in V
    visitorNames[v] = raw[v,nameCol]
end

# initialize the preference matrix 
P = zeros(Int64,n_V,n_F)

# fill out the preference matrix
for v in V
    
    # loop over each visitor's list of names
    for c = 1:10
        # extract the name corresponding to the cth choice
        curName = raw[v,firstPrefCol + c - 1]
        
        # check for a string
        if typeof(curName) == String
        
            # find the index corresponding to the current faculty member
            f = 1
            nameFound = false
            while nameFound == false && f <= n_F
                # extract the faculty name corresponding to f
                facName = facultyNames[f]

                #compare the two strings
                if contains(facName,curName) == true # a match has been found

                    #raise the nameFound flag
                    nameFound = true

                else # no match has been found

                    #increment fInd and move on
                    f = f + 1         
                end

            end # ends the while loop

            #decide why we exited the while loop
            if nameFound == true # the name was found
                # update the preference matrix
                P[v,f] = prefPoints(c)
            else #the name was not found
                visName = visitorNames[v]
                println("Warning: Visitor '$visName' requested a meeting with '$curName', but I can't find this string in the faculty list.  I will skip over this. Please double-check the spelling.\n")
            end
        else # this is an empty cell
            #do nothing for this c
        end  #ends the split checking for a valid string
        
    end
end

# Define and Solve the Optimization Problem
## Initialize the Model and Define the Decision Variables

In [ ]:
# Describe the Optimization Problem
using JuMP
using Cbc
# using Gurobi

m = Model(solver = CbcSolver())

## define the decision variables
@variable(m, x[1:n_V,1:n_F,1:n_T],Bin)
@variable(m, y[1:n_V,1:n_T],Bin)
@variable(m, h);

## Define the Constraints
Here is where you specify the minimum number of faculty meetings and minimum number of free periods for each visitor.

Be careful not to be too demanding here.
There are 8 time periods, so, for example, if you specify a minimum of 6 faculty meetings and a minimum of 3 free periods, the code will of course not be able to find a feasible schedule.
In general, the sum of the specified minimums can be at most 8.

Also, depending on the number of visitors and the availability of the faculty, it may not be possible to specify the minimum number of faculty meetings as high as you'd like.
For example, if there are 30 visitors and a total of 112 time periods that faculty are available to meet, you cannot require that each visitor has a minimum of 4 faculty meetings, because $30 \times 4 = 120 > 112$.
If you make the minimum number of required faculty meetings too high, the code will return "Infeasible".
If that happens, just reduce the minimum required faculty meetings and try again.

In [ ]:
##########################################################
# Specify the minimums
minFacultyMeetings = 4   # <-- You can modify this number
minFreePeriods = 3       # <-- You can modify this number
##########################################################

# define the constraints
## constraint 1
for v in V, t in T
   @constraint(m, sum(x[v,:,t]) + y[v,t] == 1) 
end

## constraint 2
for f in F, t in T
   @constraint(m, sum(x[:,f,t]) <= 1) 
end

## constraint 3
for f in F, t in T
   @constraint(m, sum(x[:,f,t]) <= A[f,t]) 
end

## constraint 4
for v in V, f in F
   @constraint(m, sum(x[v,f,:]) <= 1) 
end

## constraint 5
for v in V
   @constraint(m, sum(y[v,:]) >= minFreePeriods) 
end

## constraint 6
for v in V
    @constraint(m, sum(x[v,:,:]) >= minFacultyMeetings) 
end

# ## constraint 6
# maxFreePeriods = 4
# for v in V
#     @constraint(m, sum(y[v,:]) <= maxFreePeriods) 
# end

## constraint 7
for v in V
   @constraint(m, sum(x[v,f,t].*P[v,f] for f in F, t in T) >= h) 
end
    
## constraint 8
for v in V
    @constraint(m, sum(x[v,f,t] for f in F, t in T_TG) <= 1) 
end

## Define the Objective Function
The objective function has several weighting coefficients, $w_\text{pref}, w_\text{meet}$, and $w_\text{min}$.
Each of these coefficients corresponds to a particular objective.

| Weighting Coefficient | Suggested Value | Objective |
|:---------------------:|:---------------:|-----------|
| $w_\text{pref}$       | 1               | Give the visitors meetings with their preferred faculty.     |
| $w_\text{meet}$       | 1               | It's better to have a meeting with a random faculty member than no meeting at all. |
| $w_\text{min}$        | 5               | Maximize the minimum level of happiness.     |

You can set the values of these coefficients in the code below. 
Each coefficient most be nonnegative (i.e., $w_i \geq 0$).
**The greater the value of the coefficient, the harder the code will try to accomplish the corresponding objective.** That being said, if you double each coefficient, it will make no difference to the code, because it is really the *ratios* between them that matter.

Manipulating these weighting coefficients will very likely effect the summary statistics and schedule produced by the code, but small changes in the weights may have no effect.

In [ ]:
###################################################################
# specify the weighting factors for the objectives
w_pref = 1  # <-- You can modify these numbers
w_meet = 1  # <-- You can modify these numbers
w_min = 5   # <-- You can modify these numbers
###################################################################

# construct the objective coefficient matrices
c_1 = zeros(n_V,n_F,n_T)
for v in V, f in F
    c_1[v,f,:] =w_pref*P[v,f]
end

c_2 = zeros(n_V,n_F,n_T)
for v in V, f in F
    c_2[v,f,:] = w_meet
end

c_3 = w_min

c = c_1 + c_2

# define the objective function
@objective(m,Max,sum(sum(sum(c.*x))) + c_3*h);

## Solve the Optimization Problem
Running this cell could take anywhere from a few seconds to a few minutes, after which time it will display one word:

> Optimal

If the code has been running for more than 20 minutes, there's probably something wrong.  Contact Garrett.

In [ ]:
# Solve the Optimization Problem
status = solve(m)
println(status)

# extract the numerical solution
x = round(Int,getvalue(x))
y = round(Int,getvalue(y));


# Export the Results
The following two subsections construct two different outputs showing the suggested meeting schedule.
The first of these outputs, "Visitor-Focused Output.csv", shows the schedule for each visitor.
The second, "Faculty-Focused Output.csv", shows the schedule for each faculty member.
Both files will be exported to the JuliaBox directory from which you are running this code.
## Export Visitor-Focused Table

In [ ]:
# initialize the schedule
schedule = Array{String,2}(n_V,n_T)
happiness = zeros(Float64,n_V)
numMeetings = zeros(Float64,n_V)
firstChoice = Array{String,1}(n_V)
secondChoice = Array{String,1}(n_V)
thirdChoice = Array{String,1}(n_V)
fill!(firstChoice,"")
fill!(secondChoice,"")
fill!(thirdChoice,"")

#initialize the counters
firstChoiceMeetings = 0
secondChoiceMeetings = 0
thirdChoiceMeetings = 0

# loop over the visitors
for v in V
    # loop over the periods
    for t = 1:n_T
        # check for an appointment with Prof. Freetime
        if y[v,t] == 1 # visitor v is scheduled for free time during period t
            schedule[v,t] = "Prof. Freetime"
        else # look for a faculty meeting
            f = findfirst(x[v,:,t])
            if f == 0
                println("Warning: Student $v was not scheduled for a meeting or free time during period $t.\n")
            else 
                # put the faculty member f's name in the visitor's schedule
                schedule[v,t] = facultyNames[f]
                
                # increment the number of meetings
                numMeetings[v] = numMeetings[v] + 1
                
                # check for first choice
                if c_1[v,f,t] == prefPoints(1)
                    firstChoice[v] = "Yes"
                    firstChoiceMeetings = firstChoiceMeetings + 1
                elseif c_1[v,f,t] == prefPoints(2)
                    secondChoice[v] = "Yes"
                    secondChoiceMeetings = secondChoiceMeetings + 1
                elseif c_1[v,f,t] == prefPoints(3)
                    thirdChoice[v] = "Yes"
                    thirdChoiceMeetings = thirdChoiceMeetings + 1
                end
            end
            
        end
    end
    
    #compute this visitor's happiness
    happiness[v] = sum(sum((c[v,:,:]+w_meet).*x[v,:,:]))/(sum(prefPoints(i) for i = 1:(n_T - minFreePeriods)) + w_meet*(n_T - minFreePeriods))
    happiness[v] = sum(sum((c[v,:,:]+w_meet).*x[v,:,:]))
end

In [ ]:
# write the results to Excel
using DataArrays, DataFrames
visitorSchedule = DataFrame(Visitor_Name = visitorNames, Meeting_1 = schedule[:,1], Meeting_2 = schedule[:,2], Meeting_3 = schedule[:,3], Meeting_4 = schedule[:,4], Meeting_5 = schedule[:,5], Meeting_6 = schedule[:,6], Meeting_7 = schedule[:,7], Meeting_8 = schedule[:,8], Happiness_Score = happiness, First_Choice = firstChoice, Second_Choice = secondChoice, Third_Choice = thirdChoice)
writetable("Visitor-Focused Output.csv", visitorSchedule)

## Export the Faculty-Focused Table

In [ ]:
# initialize the schedule
schedule = Array{String,2}(n_F,n_T)
fill!(schedule,"")

# loop over the faculty
for f in F
    
    # loop over the time periods
    for t in T
        
        # look for a meeting with a student
        v = findfirst(x[:,f,t])
        
        if v == 0 # no meeting was found
            # check the faculty's availability
            if A[f,t] == 1 # the faculty is available during this time slot
                #indicate that this slot is open in the faculty's schedule
                schedule[f,t] = "FREE"
            end
        else # a meeting was found
            # fill in the appropriate cell of the schedule
            schedule[f,t] = visitorNames[v]
        end
        
    end
end

# write the results to Excel
facultySchedule = DataFrame(Faculty = facultyNames, Meeting_1 = schedule[:,1], Meeting_2 = schedule[:,2], Meeting_3 = schedule[:,3], Meeting_4 = schedule[:,4], Meeting_5 = schedule[:,5], Meeting_6 = schedule[:,6], Meeting_7 = schedule[:,7], Meeting_8 = schedule[:,8])
writetable("Faculty-Focused Output.csv", facultySchedule)

# Calculate some Statistics
This code prints some statistics that give you a sense of the quality of the schedule generated.

In [ ]:
# calculate some fractions
firstChoiceFraction = firstChoiceMeetings/n_V
secondChoiceFraction = secondChoiceMeetings/n_V
thirdChoiceFraction = thirdChoiceMeetings/n_V

minHappinessScore = round(minimum(happiness),2)
avgHappinessScore = round(mean(happiness),2)
maxHappinessScore = round(maximum(happiness),2)

minMeetings = minimum(numMeetings)
avgMeetings = round(mean(numMeetings),2)
maxMeetings = maximum(numMeetings)


# print the results
println("The fraction of visitors who will meet with their first choice is: $firstChoiceFraction")
println("The fraction of visitors who will meet with their second choice is: $secondChoiceFraction")
println("The fraction of visitors who will meet with their third choice is: $thirdChoiceFraction")
println("")
println("The average happiness score is: $avgHappinessScore")
println("The minimum happiness score is: $minHappinessScore")
println("The maximum happiness score is: $maxHappinessScore")
println("")
println("The average number of meetings is: $avgMeetings")
println("The minimum number of meetings is: $minMeetings")
println("The maximum number of meetings is: $maxMeetings")

# The End
If you like what you see, go back to the other tab in your browser which shows the JuliaBox file directory.
From there, you can download the two output files, following the instructions in the "Using the ChemE Visit Weekend Scheduling Code" document.

If you don't like what you see, return to the instructions at the top.